In [1]:
import gym
from gym_power_trading.envs import PowerTradingEnv
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

In [2]:
file_name = 'data\AEP_PSGC1_AMP_long.h5'
path = Path(file_name).resolve()
uri = str(path)

In [3]:
df = pd.read_hdf(uri)
df.head()

HE
MARKET_DAY NODE          TYPE    VALUE HourEnding       
2021-07-01 AEP.PSGC1.AMP Gennode LMP   1           25.88
                                       2           24.88
                                       3           23.45
                                       4           22.47
                                       5           21.73

In [3]:
my_dataframe = pd.read_csv("RandomCloseValues.csv")

In [4]:
env = PowerTradingEnv(df=my_dataframe, window_size=3, frame_bound=(10, 100))
obs = env.reset()
for _ in tqdm(range(100)):
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
    print(info)
    if done:
        obs = env.reset()

Discrete(3)


 89%|████████▉ | 89/100 [00:00<00:00, 44450.23it/s]

{'total_reward': 0.0, 'total_profit': 1.0, 'position': 2, 'battery_charge': 0}
{'total_reward': 4.381276983963815, 'total_profit': 1.0, 'position': 1, 'battery_charge': 20}
{'total_reward': 38.135878211573555, 'total_profit': 1.0, 'position': 1, 'battery_charge': 40}
{'total_reward': -697.9844361355431, 'total_profit': -731.439712775381, 'position': 0, 'battery_charge': 20}
{'total_reward': -1430.9456852360777, 'total_profit': -1460.7361556304131, 'position': 0, 'battery_charge': 0}
{'total_reward': -1430.9456852360777, 'total_profit': -1460.7361556304131, 'position': 0, 'battery_charge': 0.0}
{'total_reward': -1430.9456852360777, 'total_profit': -1460.7361556304131, 'position': 0, 'battery_charge': 0.0}
{'total_reward': -1430.9456852360777, 'total_profit': -1460.7361556304131, 'position': 0, 'battery_charge': 0.0}
{'total_reward': -1430.9456852360777, 'total_profit': -1460.7361556304131, 'position': 0, 'battery_charge': 0.0}
{'total_reward': -1430.9456852360777, 'total_profit': -1460.

IndexError: index 93 is out of bounds for axis 0 with size 93